# Guided Project : Building a Database for Crime Reports

## Introduction

The goal of this guided project is to build a database for storing data related with crimes that occured in Boston. This dataset is available in the file `boston.csv`.

We will create a database named `crimes_db` with a table named `boston_crimes` with appropriate datatypes for storing the data from the `boston.csv` file. We will create the table inside a schema named `crimes`.
We will also create the `readonly` and `readwrite` groups with the appropriate privileges as we have discussed in this course.
Finally, we will also create one user for each of these groups.

The following diagram illustrates a high level overview of what we want to acheive:
![title](img/diagram.png)

## Creating the Database and the Schema

First, we start by creating a databse for storing our crime data.

When we connect to the Postgres server, we need to specify a database name. Since the `crime_db` database does not exist yet, we can create it by connecting to Dataquest's `dq` database.

In [2]:
import psycopg2

conn = psycopg2.connect(dbname="dq", user="dq")

# Set autocommit to True as required for databse creation 
conn.autocommit = True
cur = conn.cursor()

# Creation of the crime_db database
cur.execute("CREATE DATABASE crime_db;")

conn.close()

ProgrammingError: database "crime_db" already exists


Then, we create a schema in this new database to contain the tables.

In [3]:
conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True
cur = conn.cursor()

# Creation of the crimes schema
cur.execute("CREATE SCHEMA crimes;")

ProgrammingError: schema "crimes" already exists


## Analyzing the Dataset

We now have a database and a schema - we are ready to start creating tables. But before we do that, let's gather some data about our crime dataset so that we can more easily select the right datatypes to use in our table.

Let's start by reading the column names from the `boston.csv` file as well as the first row. In this way we will have them at hand throughout this guided project so that we can easily take a look at them at any moment.

In [4]:
import csv

with open("boston.csv", "r") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)
    
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


The first column represents the identifier of the crime. The second contains a numeric identifier code for the committed crime. The third represents a description of the crime. The next two rows contain the date on which the crime happened and the corresponding day of the week. Finally, the last two columns represent the location of the crime with a latitude and longitude coordinates.

Before we create a table for storing the crime data, we need to identify the proper datatypes for the columns. To help us with that, let's create a function — `get_col_set()` — that given the name of a CSV file and a column index (starting at 0) that computes a Python set with all distinct values contained in that column. 

This function will be useful for two reasons:
1. Checking whether an enumerated datatype might be a good choice for representing a column.
2. Computing the maximum length of any text-like column to select appropriate sizes for `VARCHAR` columns.

Let's compute the number of different values in each column of the `boston.csv` file.

In [6]:
def get_col_set(csv_filename, col_index):
    
    import csv
    values = set()
    with open("boston.csv", "r") as file:
        reader = csv.reader(file)
        for row in reader:
            values.add(row[col_index])
    return values

We can now compute the number of different values each of the seven columns of our dataset contain.

In [7]:
for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298330
offense_code	220
description	240
date	1178
day_of_the_week	8
lat	18178
long	18178


Columns with a low number of values, like the `day_of_the_week` column, tend to be good candidates for enumerated datatypes.

Another important aspect is to know the longest word in any column containing textual data. We can actually use the previous function for computing this:
1. Use the `get_col_set()` function to get the distinct values of that column into a variable named `col_values`
2. Iterate over the elements of `col_values` and find the one which is the longest

There are two textual column in the data set, namely, the `description` and `day_of_the_week` columns. However the day of the week contains only 7 values, one for each day. We can tell that the longest of them is `Wednesday` without needing any computation.

Let's compute the maximum length of each value in the `description` column.

In [8]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [9]:
descriptions = get_col_set("boston.csv", 2)

max_length = 0

for description in descriptions:
    max_length = max(max_length, len(description))

print(max_length)

58


The longest element of the `description` column contains 58 characters.

In the next section, we use this information to select the appropriate datatypes while creating our table.

## Creating the Table

We now create a table named `boston_crimes` inside the `crimes` schema of the `crime_db` database.

First, we need to agree on the appropriate datatypes.

In [10]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


Looking at the first row of the dataset, we can see the following datatypes:
- `incident_number` : integer
- `offense_code` : integer
- `description` : text
- `date` : date
- `day_of_the_week` : text
- `lat` : decimal
- `long` : decimal

Only the column `day_of_the_weekend` has a small enough range of unique values to consider an enumerate datatype.

As we computed in the previous section, the longest text element of the `description` column is of size 58.
To be on the safe side, we will limit the size of `description` to 100 characters and use the VARCHAR(100) datatype.

The `lat` and `long` columns need to hold a lot of precision so we will use the decimal type for them.

In [11]:
# Creation of the enumerated datatype for weekday
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');""")

# Creation of the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes(
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100)
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

ProgrammingError: type "weekday" already exists


## Loading the Data

Now that we have created the table, we can load the data into it.

In [12]:
with open("boston.csv", "r") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER",f)
    
cur.execute("SELECT * FROM crimes.boston_crimes")
print(len(cur.fetchall()))

ProgrammingError: relation "crimes.boston_crimes" does not exist


## Handling Users and Access Rights

It is now time to handle users rights. Our goal is to create two users groups: `readonly` and `readwrite`. By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the `public` group and on the `public` schema.

### Revoking Public Priviledges

We revoke all privileges of the `public` group on the `public` schema to ensure that users will not inherit priviledges on that schema such as the ability to create tables in the `public` schema.

We also need to revoke all priviledges in the newly created schema.

In [14]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public")

### Creating Users Groups

Next, we create the `readonly` and `readwrite` users groups. Both groups shall be granted the ability to connect to the database `crimes_db` and to use the `crimes` schema.

The `readonly` group shall only have the priviledges to perform `SELECT` queries on all the tables of the `crimes` schema.

The `readwrite` group shall be able to perform `SELECT`, `INSERT`, `DELETE`, `UPDATE` queries on all the tables of the `crimes` schema.

In [16]:
cur.execute("CREATE GROUP readonly NOLOGIN")
cur.execute("CREATE GROUP readwrite NOLOGIN")

cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readonly")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readwrite")

cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite")

cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite")

### Creating One User For Each Group

The only thing left to do is to create users. We will create one user for each group and assign them to their respective group.

In [17]:
cur.execute("CREATE data_analyst WITH PASSWORD 'secret1'")
cur.execute("CREATE data_scientist WITH PASSWORD 'secret2'")

cur.execute("GRANT readonly TO data_analyst")
cur.execute("GRANT readwrite TO data_scientist")

ProgrammingError: syntax error at or near "data_analyst"
LINE 1: CREATE data_analyst WITH PASSWORD 'secret1'
               ^


We have now finished setting up our Postgres database! This database should look like the diagram shown in the introduction section.

Now, it is a good practice to test everything is configured as expected.We can query Postgres internal tables to see this information.

## Testing the Database Setup

In this last section, we test the database setup using SQL queries.

We can query the `pg_roles` table to inspect priviledges related to the database. For that, we will look at the following columns:
- `rolname`: The name of the user / group that the privilege refers to.
- `rolsuper`: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
- `rolcreaterole`: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
- `rolcreatedb`: Whether user / group can create databases. It should be False on every user / group that we have created.
- `rolcanlogin`: Whether user / group can log in. It should be True on the users and False on the groups that we have created.

We can also query the `information_schema.table_privileges` table to inspect table priviledges. We will list the privileges of each group that we have created.

In [18]:
# Close the old connection to test with a brand new one
conn.close()

# Open a new connection and create a new cursor object
conn = psycopg2.connect(dbname="crimes_db", user="dq")
cur = conn.cursor()

# Check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin
    FROM pg_roles
    WHERE rolname IN ('readonly','readwrite','data_analyst','data_scientist')
""")

for user in cur:
    print(user)
print()

# Check priviledges
cur.execute("""
    SELECT grantee, priviledge_type
    FROM information_schema.table_priviledges
    WHERE grantee IN ('readonly', 'readwrite')
""")

for user in cur:
    print(user)
    
conn.close()

OperationalError: FATAL:  database "crimes_db" does not exist
